In [1]:
import pandas as pd
from pdfminer.high_level import extract_text
import re


In [27]:
# Step 1: Read the PDF file using pdfminer to handle the text extraction
file_path_new = 'the governance of China vol 4.pdf'
text_new = extract_text(file_path_new)


In [28]:
# Function to clean text and remove page numbers
def clean_text(text):
    # Remove standard page numbers (digits at start/end of lines)
    text = re.sub(r'^\s*\d+\s*|\s*\d+\s*$', '', text, flags=re.MULTILINE)
    
    # Remove page numbers that might appear in the middle of text
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
    
    # Remove consecutive newlines (more than 2) to clean up spacing
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    return text.strip()

# Revised regular expression to match dates
pattern = re.compile(r"（\d{4}年\s?\d{1,2}月\s?\d{1,2}日）")

# Split the text using the pattern
sections = pattern.split(text_new)
titles_and_dates = pattern.findall(text_new)

# Initialize a dictionary to store speeches
speeches = {
    "title": [],
    "date": [],
    "text": [],
    "references": []
}

# Iterate through each section and date
for i, title_date in enumerate(titles_and_dates):
    # Extract the title before the date and trim excess lines
    start_index = text_new.find(title_date)
    title_text = text_new[:start_index].strip().split('\n')[-1]
    
    # Ensure we get the main text of the speech
    full_text = sections[i + 1].strip()
    
    # Clean the text to remove page numbers
    full_text = clean_text(full_text)
    
    # Split references if they exist
    ref_start = full_text.find("注释")
    if ref_start != -1:
        main_text = clean_text(full_text[:ref_start])
        references = clean_text(full_text[ref_start:])
    else:
        main_text = full_text
        references = ""
    
    # Append the extracted components to the dictionary
    speeches["title"].append(f"{title_text} (Speech {i + 1})")
    speeches["date"].append(title_date)
    speeches["text"].append(main_text)
    speeches["references"].append(references)

# Create a DataFrame with the extracted data
df_speeches = pd.DataFrame(speeches)

df_speeches.head()

,title,date,text,references
0,在庆祝中国共产党成立100周年大会上的讲话 (Speech 1),（2021年7月1日）,同志们，朋友们：\n\n今天，在中国共产党历史上，在中华民族历史上，都是一个十分 \n\n重...,注释\n\n〔1〕“五位一体”总体布局，指中国特色社会主义事业总体布局， \n\n 包括经济...
1,续写马克思主义中国化时代化新篇章※ (Speech 2),（2022年1月11日）,※这是习近平在省部级主要领导干部学习贯彻党的十九届六中全会精神专 \n\n题研讨班开班式上的...,
2,新时代党和人民奋进的必由之路※ (Speech 3),（2022年3月5日）,※这是习近平在参加十三届全国人大五次会议内蒙古代表团审议时的讲话要点。\n\n回顾新时代党和...,注释\n\n〔1〕见东汉王充《论衡·别通》。\n\n不断提高政治判断力、政治领悟力、政治执行...
3,加强党对社会主义现代化建设的全面领导※ (Speech 4),（2021年1月11日）,※这是习近平在省部级主要领导干部学习贯彻党的十九届五中全会精神专 \n\n题研讨班上讲话的一...,注释\n\n〔1〕见邓小平《在武昌、深圳、珠海、上海等地的谈话要点》（《邓 \n\n小平文选...
4,坚持党的政治建设，始终保持党的团结统一※ (Speech 5),（2021年11月11日）,※这是习近平在中共十九届六中全会第二次全体会议上讲话的一部分。\n\n保证党的团结统一是党的...,


In [29]:
# Save the DataFrame to a CSV file
df_speeches.to_csv("governance of China vol 4.csv", index=False, encoding="utf-8-sig")

In [30]:
# Load each CSV file into a DataFrame
vol1 = pd.read_csv("governance of china vol 1.csv", encoding="utf-8-sig")
vol2 = pd.read_csv("governance of china vol 2.csv", encoding="utf-8-sig")
vol3 = pd.read_csv("governance of china vol 3.csv", encoding="utf-8-sig")
vol4 = pd.read_csv("governance of china vol 4.csv", encoding="utf-8-sig")

# Merge all DataFrames into one
merged_data = pd.concat([vol1, vol2, vol3, vol4], ignore_index=True)


In [32]:
merged_data.head()
# show the number of rows and columns in the dataframe
#print(merged_data.shape)



,title,date,text,references
0,就是我们的奋斗目标 * (Speech 1),（2012年11月15日）,* 这是习近平在十八届中央政治局常委同中外记者见面时讲话的主要部分。\n\n记者朋友们对十八...,NaN
1,学习宣传贯彻党的十八大精神 * (Speech 2),（2012年11月17日）,* 这是习近平在主持十八届中央政治局第一次集体学习 〔1〕 时的讲话。\n\n党的十八大报告...,注 释\n\n〔1〕 中央政治局集体学习，指中共中央政治局定期学习制度。由中共中央总书记主持...
2,发展中国特色社会主义 * (Speech 3),（2013年1月5日）,* 这是习近平在新进中央委员会的委员、候补委员学习贯彻党的十八大精神研讨班上的讲 \n\n话...,注 释\n\n〔1〕 科学社会主义，广义上指马克思主义的整个思想体系；狭义上的科学社会主义是...
3,毛泽东思想活的灵魂 * (Speech 4),（2013年12月26日）,* 这是习近平在纪念毛泽东同志诞辰120周年座谈会上讲话的一部分。\n\n毛泽东思想活的灵魂...,注 释\n\n〔1〕 见毛泽东《改造我们的学习》（《毛泽东选集》第3卷，人民出版社1991年...
4,最伟大的梦想 * (Speech 5),（2012年11月29日）,* 这是习近平在参观 《复兴之路》展览时的讲话。\n\n《复兴之路》这个展览，回顾了中华民族...,注 释\n\n〔1〕 见毛泽东《忆秦娥·娄山关》（《毛泽东诗词集》，中央文献出版社1996年...


In [33]:
# Function to convert the date string to the "year-month-day" format
def convert_date(date_str):
    try:
        # Clean up the date string by stripping extra spaces
        date_str = date_str.replace(" ", "")
        # Extract the year, month, and day using slicing
        year = date_str[1:5]
        month = date_str[6:date_str.find("月")].zfill(2)  # Pad single-digit months with a leading zero
        day = date_str[date_str.find("月") + 1:date_str.find("日")].zfill(2)  # Pad single-digit days with a leading zero
        return f"{year}-{month}-{day}"
    except Exception:
        return None  # Return None if there's an issue with parsing

# Apply the conversion function to create a new 'formatted_date' column
merged_data["formatted_date"] = merged_data["date"].apply(convert_date)

# Convert the 'formatted_date' column to datetime and extract the year
merged_data['formatted_date'] = pd.to_datetime(merged_data['formatted_date'], errors='coerce')  # Use 'coerce' to handle errors
merged_data['year'] = merged_data['formatted_date'].dt.year

# Save the updated DataFrame to a new CSV file
merged_data.to_csv("governance_of_china_all_volumes_with_formatted_date.csv", index=False, encoding="utf-8-sig")

merged_data.head()

,title,date,text,references,formatted_date,year
0,就是我们的奋斗目标 * (Speech 1),（2012年11月15日）,* 这是习近平在十八届中央政治局常委同中外记者见面时讲话的主要部分。\n\n记者朋友们对十八...,NaN,2012-11-15,2012
1,学习宣传贯彻党的十八大精神 * (Speech 2),（2012年11月17日）,* 这是习近平在主持十八届中央政治局第一次集体学习 〔1〕 时的讲话。\n\n党的十八大报告...,注 释\n\n〔1〕 中央政治局集体学习，指中共中央政治局定期学习制度。由中共中央总书记主持...,2012-11-17,2012
2,发展中国特色社会主义 * (Speech 3),（2013年1月5日）,* 这是习近平在新进中央委员会的委员、候补委员学习贯彻党的十八大精神研讨班上的讲 \n\n话...,注 释\n\n〔1〕 科学社会主义，广义上指马克思主义的整个思想体系；狭义上的科学社会主义是...,2013-01-05,2013
3,毛泽东思想活的灵魂 * (Speech 4),（2013年12月26日）,* 这是习近平在纪念毛泽东同志诞辰120周年座谈会上讲话的一部分。\n\n毛泽东思想活的灵魂...,注 释\n\n〔1〕 见毛泽东《改造我们的学习》（《毛泽东选集》第3卷，人民出版社1991年...,2013-12-26,2013
4,最伟大的梦想 * (Speech 5),（2012年11月29日）,* 这是习近平在参观 《复兴之路》展览时的讲话。\n\n《复兴之路》这个展览，回顾了中华民族...,注 释\n\n〔1〕 见毛泽东《忆秦娥·娄山关》（《毛泽东诗词集》，中央文献出版社1996年...,2012-11-29,2012
